Implementacja

In [ ]:
# import numpy as np
# import random
# import sys


# # Disjoint set data structure
# class DSU:
#     def __init__(self, n):
#         self.parent = list(range(n))
#         self.rank = [1] * n

#     def find(self, i):
#         if self.parent[i] != i:
#             self.parent[i] = self.find(self.parent[i])
#         return self.parent[i]

#     def union(self, x, y):
#         s1 = self.find(x)
#         s2 = self.find(y)
#         if s1 != s2:
#             if self.rank[s1] < self.rank[s2]:
#                 self.parent[s1] = s2
#             elif self.rank[s1] > self.rank[s2]:
#                 self.parent[s2] = s1
#             else:
#                 self.parent[s2] = s1
#                 self.rank[s1] += 1

# '''Generuje prosty graf pełny o n wierzchołkach, z wagami krawędzi
# pochodzącymi z rozkładu dist z parametrami *params.
# Dostępne rozkłady: 
# 'uniform'       -       rozkład jednostajny;        *params: a, b - krańce przedziału
# 'beta'          -       rozkład beta;               *params: alpha, beta
# 'gamma'         -       rozkład gamma;              *params: alpha, beta
# Zwraca macierz sąsiedztwa grafu z jego wagami.'''
# class Graph():
#     def __init__(self, vertices, dist, *params):
#         self.V = vertices
#         self.E = self.graph_rand_weights(vertices, dist, *params)

#     def graph_rand_weights(self, n, dist, *params):
#         E = np.zeros((n, n), dtype=np.float64)
#         for i in range(n):
#             for j in range(i + 1, n):
#                 if dist == 'uniform':
#                     w = random.uniform(*params)
#                 elif dist == 'beta':
#                     w = random.betavariate(*params)
#                 elif dist == 'gamma':
#                     w = random.gammavariate(*params)
#                 else:
#                     raise ValueError('Unrecognized distribution')
#                 E[i,j] = E[j,i] = w
#         return E

#     # A utility function to print 
#     # the constructed MST stored in parent[]
#     def printPrimMST(self, parent):
#         print("Edge \tWeight")
#         for i in range(1, self.V):
#             print(parent[i], "-", i, "\t", self.E[parent[i]][i])

#     def printPrimMSTvalue(self, parent):
#         value = 0
#         for i in range(1, self.V):
#             value += self.E[parent[i]][i]
#         print("MST value:", value)



#     # A utility function to find the vertex with
#     # minimum distance value, from the set of vertices
#     # not yet included in shortest path tree
#     def minKey(self, key, mstSet):

#         # Initialize min value
#         min = sys.maxsize

#         for v in range(self.V):
#             if key[v] < min and mstSet[v] == False:
#                 min = key[v]
#                 min_index = v

#         return min_index

#     # Function to construct and print MST for a graph
#     # represented using adjacency matrix representation
#     def primMST(self):

#         # Key values used to pick minimum weight edge in cut
#         key = [sys.maxsize] * self.V
#         parent = [None] * self.V  # Array to store constructed MST
#         # Make key 0 so that this vertex is picked as first vertex
#         key[0] = 0
#         mstSet = [False] * self.V

#         parent[0] = -1  # First node is always the root of

#         for cout in range(self.V):

#             # Pick the minimum distance vertex from
#             # the set of vertices not yet processed.
#             # u is always equal to src in first iteration
#             u = self.minKey(key, mstSet)

#             # Put the minimum distance vertex in
#             # the shortest path tree
#             mstSet[u] = True

#             # Update dist value of the adjacent vertices
#             # of the picked vertex only if the current
#             # distance is greater than new distance and
#             # the vertex in not in the shortest path tree
#             for v in range(self.V):

#                 # graph[u][v] is non zero only for adjacent vertices of m
#                 # mstSet[v] is false for vertices not yet included in MST
#                 # Update the key only if graph[u][v] is smaller than key[v]
#                 if self.E[u][v] > 0 and mstSet[v] == False \
#                 and key[v] > self.E[u][v]:
#                     key[v] = self.E[u][v]
#                     parent[v] = u

#         self.printPrimMSTvalue(parent)

#     def kruskals_mst(self):

#         # Sort all edges
#         edges = []
#         for i in range(self.V):
#             for j in range(i + 1, self.V):
#                 edges.append((i, j, self.E[i][j]))
#         edges = sorted(edges, key=lambda x: x[2])

#         # Traverse edges in sorted order
#         dsu = DSU(self.V)
#         cost = 0
#         count = 0
#         for x, y, w in edges:

#             # Make sure that there is no cycle
#             if dsu.find(x) != dsu.find(y):
#                 dsu.union(x, y)
#                 cost += w
#                 count += 1
#                 if count == self.V - 1:
#                     break
#         return cost


In [ ]:
# G = Graph(500, 'beta', 0.5, 0.5)

# print(G.E)
# G.primMST()
# G.kruskals_mst()

[[0.00000000e+00 8.70087800e-01 5.23819533e-01 ... 9.09874066e-01
  2.90883168e-01 3.77503105e-05]
 [8.70087800e-01 0.00000000e+00 5.41741129e-01 ... 1.95503746e-01
  9.17678799e-01 1.05767742e-01]
 [5.23819533e-01 5.41741129e-01 0.00000000e+00 ... 3.27064392e-01
  5.38221095e-01 2.16413804e-04]
 ...
 [9.09874066e-01 1.95503746e-01 3.27064392e-01 ... 0.00000000e+00
  4.99037695e-01 9.65205281e-02]
 [2.90883168e-01 9.17678799e-01 5.38221095e-01 ... 4.99037695e-01
  0.00000000e+00 9.05267730e-01]
 [3.77503105e-05 1.05767742e-01 2.16413804e-04 ... 9.65205281e-02
  9.05267730e-01 0.00000000e+00]]
MST value: 0.011004264301656963


0.011004264301656977

Wykresy

In [ ]:
import time
import random
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from heapq import heappush, heappop

# Jeśli masz już DSU i Graph z def. w Twoim kodzie, możesz użyć ich.
# Tutaj zakładam minimalną niezależność i definiuję DSU / generator grafów lokalnie,
# ale możesz też korzystać z istniejącej klasy Graph.

class DSU:
  def __init__(self, n):
    self.parent = list(range(n))
    self.rank = [1] * n

  def find(self, i):
    if self.parent[i] != i:
      self.parent[i] = self.find(self.parent[i])
    return self.parent[i]

  def union(self, x, y):
    s1 = self.find(x)
    s2 = self.find(y)
    if s1 != s2:
      if self.rank[s1] < self.rank[s2]:
        self.parent[s1] = s2
      elif self.rank[s1] > self.rank[s2]:
        self.parent[s2] = s1
      else:
        self.parent[s2] = s1
        self.rank[s1] += 1

'''Generuje prosty graf pełny o n wierzchołkach, z wagami krawędzi
pochodzącymi z rozkładu dist z parametrami *params.
Dostępne rozkłady: 
'uniform'       -       rozkład jednostajny;        *params: a, b - krańce przedziału
'beta'          -       rozkład beta;               *params: alpha, beta
'gamma'         -       rozkład gamma;              *params: alpha, beta
Zwraca macierz sąsiedztwa grafu z jego wagami.'''
def gen_full_graph_matrix(n, dist='uniform', *params, rng=None):
  if rng is None:
    rng = random
  E = np.zeros((n, n), dtype=np.float64)
  for i in range(n):
    for j in range(i+1, n):
      if dist == 'uniform':
        w = rng.uniform(*params)
      elif dist == 'beta':
        w = rng.betavariate(*params)
      elif dist == 'gamma':
        w = rng.gammavariate(*params)
      else:
        raise ValueError('Unrecognized distribution')
      E[i, j] = E[j, i] = w
  return E

# Prim O(n^2) zwracający koszt MST
def prim_mst_cost(E):
  n = E.shape[0]
  in_mst = [False] * n
  key = [float('inf')] * n
  key[0] = 0.0
  parent = [-1] * n
  for _ in range(n):
    # wybierz wierzchołek z minimalnym key spoza MST
    u = -1
    best = float('inf')
    for v in range(n):
      if not in_mst[v] and key[v] < best:
        best = key[v]
        u = v
    if u == -1:
      break
    in_mst[u] = True
    # aktualizuj klucze
    for v in range(n):
      if not in_mst[v] and E[u, v] > 0 and E[u, v] < key[v]:
        key[v] = E[u, v]
        parent[v] = u
  # oblicz koszt
  cost = 0.0
  for v in range(1, n):
    if parent[v] != -1:
      cost += E[v, parent[v]]
  return cost

# Kruskal (sortujemy wszystkie krawędzie)
def kruskal_mst_cost(E):
  n = E.shape[0]
  edges = []
  for i in range(n):
    for j in range(i+1, n):
      edges.append((E[i, j], i, j))
  edges.sort(key=lambda x: x[0])
  dsu = DSU(n)
  cost = 0.0
  taken = 0
  for w, u, v in edges:
    if dsu.find(u) != dsu.find(v):
      dsu.union(u, v)
      cost += w
      taken += 1
      if taken == n - 1:
        break
  return cost

# --- Parametry eksperymentu (dostosuj) ---
n_min = 20
n_max = 2000
step = 10
reps = 100

distributions = [
  ('uniform', (0.0, 1.0)),
  # ('gamma', (2.0, 1.0)),
  ('gamma', (math.pi, math.e)),
  ('beta', (math.pi, math.e))
]

# dla powtarzalności:
SEED = 12345
random.seed(SEED)
np.random.seed(SEED)

# miejsce na wyniki
rows = []

total_runs = len(distributions) * len(range(n_min, n_max+1, step)) * reps
run_i = 0

for dist_name, params in distributions:
  for n in range(n_min, n_max+1, step):
    for rep in range(reps):
      run_i += 1
      # generuj graf
      E = gen_full_graph_matrix(n, dist_name, *params, rng=random)
      # Prim
      t0 = time.perf_counter()
      prim_cost = prim_mst_cost(E)
      t1 = time.perf_counter()
      prim_time = t1 - t0
      # Kruskal
      t0 = time.perf_counter()
      kruskal_cost = kruskal_mst_cost(E)
      t1 = time.perf_counter()
      kruskal_time = t1 - t0

      rows.append({
        'dist': dist_name,
        'n': n,
        'rep': rep,
        'prim_time': prim_time,
        'kruskal_time': kruskal_time,
        'prim_cost': prim_cost,
        'kruskal_cost': kruskal_cost
      })

      if run_i % 100 == 0:
        print(f'Progress: {run_i}/{total_runs} runs done')

df = pd.DataFrame(rows)

# Statystyki (dla każdego (dist, n, alg) )
def summarize_times_and_costs(df):
  groups = []
  for dist_name in df['dist'].unique():
    for n in sorted(df.loc[df['dist'] == dist_name, 'n'].unique()):
      sub = df[(df['dist'] == dist_name) & (df['n'] == n)]
      for alg in ['prim', 'kruskal']:
        times = sub[f'{alg}_time']
        costs = sub[f'{alg}_cost']
        groups.append({
          'dist': dist_name,
          'n': n,
          'alg': alg,
          'time_mean': times.mean(),
          'time_std': times.std(ddof=0),
          'time_min': times.min(),
          'time_max': times.max(),
          'time_cv': times.std(ddof=0) / times.mean() if times.mean() > 0 else np.nan,
          'cost_mean': costs.mean(),
          'cost_std': costs.std(ddof=0),
          'cost_min': costs.min(),
          'cost_max': costs.max()
        })
  return pd.DataFrame(groups)

summary_df = summarize_times_and_costs(df)
print(summary_df.head())

# --- Wykresy ---
sns.set(style="whitegrid")

# 1) Czasy: średni czas vs n (z paskiem std) dla obu algorytmów i dla każdego rozkładu
for dist_name in df['dist'].unique():
  plt.figure(figsize=(8, 5))
  sub = summary_df[summary_df['dist'] == dist_name]
  sns.lineplot(data=sub, x='n', y='time_mean', hue='alg', marker='o')
  # pokaż standard deviation jako wypełnienie ręcznie
  for alg in ['prim', 'kruskal']:
    alg_sub = sub[sub['alg'] == alg]
    plt.fill_between(alg_sub['n'],
                     alg_sub['time_mean'] - alg_sub['time_std'],
                     alg_sub['time_mean'] + alg_sub['time_std'],
                     alpha=0.2)
  plt.title(f'Average runtime vs n (dist={dist_name})')
  plt.xlabel('n (liczba wierzchołków)')
  plt.ylabel('czas [s]')
  plt.legend(title='alg')
  plt.tight_layout()
  plt.savefig(f'runtime_vs_n_{dist_name}.png', dpi=150)
  plt.show()

# 2) Histogram / KDE rozkładu wag MST dla jednej wybranej wielkości n (np. najwyższe n)
for dist_name in df['dist'].unique():
  max_n = df[df['dist'] == dist_name]['n'].max()
  sub = df[(df['dist'] == dist_name) & (df['n'] == max_n)]
  plt.figure(figsize=(8,5))
  sns.kdeplot(sub['prim_cost'], label='Prim', fill=True)
  sns.kdeplot(sub['kruskal_cost'], label='Kruskal', fill=True)
  plt.title(f'Distribution of MST weight for n={max_n}, dist={dist_name}')
  plt.xlabel('MST weight')
  plt.legend()
  plt.tight_layout()
  plt.savefig(f'mst_weight_dist_n{max_n}_{dist_name}.png', dpi=150)
  plt.show()

# Zapis wyników do csv
df.to_csv('mst_experiment_raw.csv', index=False)
summary_df.to_csv('mst_experiment_summary.csv', index=False)

print('Wszystkie wyniki zapisane: mst_experiment_raw.csv i mst_experiment_summary.csv')


Progress: 100/59700 runs done
Progress: 200/59700 runs done
Progress: 300/59700 runs done
Progress: 400/59700 runs done
Progress: 500/59700 runs done
Progress: 600/59700 runs done
Progress: 700/59700 runs done
Progress: 800/59700 runs done
Progress: 900/59700 runs done
Progress: 1000/59700 runs done
Progress: 1100/59700 runs done
Progress: 1200/59700 runs done
Progress: 1300/59700 runs done
Progress: 1400/59700 runs done
Progress: 1500/59700 runs done
Progress: 1600/59700 runs done
Progress: 1700/59700 runs done
Progress: 1800/59700 runs done
Progress: 1900/59700 runs done
Progress: 2000/59700 runs done
Progress: 2100/59700 runs done
Progress: 2200/59700 runs done
Progress: 2300/59700 runs done
Progress: 2400/59700 runs done
Progress: 2500/59700 runs done
Progress: 2600/59700 runs done
Progress: 2700/59700 runs done
Progress: 2800/59700 runs done
Progress: 2900/59700 runs done
Progress: 3000/59700 runs done
Progress: 3100/59700 runs done
Progress: 3200/59700 runs done
Progress: 3300/59